In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%pylab inline

#В файле размещены данные обо всех сессиях всех игроков игры X за последние 90 дней 
#(udid - id игрока, time - дата, count - количество сессий в день, avg_duration - средняя продолжительность сессии 
#в секундах). Постройте на основании этих данных модель прогнозирования ухода игрока. Применяемые методы и 
#инструменты ограничены лишь вашей фантазией и умениями. Главное, что модель в итоге должна уметь прогнозировать 
#для каждого конкретного игрока с какой-либо вероятностью то, что он в ближайшее время покинет проект. 
#Покинувшим проект игроком давайте считать того, кто не заходил в игру уже минимум две недели. 


Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('TestGameSessions.csv')
df.head()

,udid,time,count,avg_duration
0,104008439,1474588800,2,112
1,104011104,1474588800,2,44
2,104022241,1474588800,2,413
3,104022356,1474588800,2,125
4,104022069,1474592400,5,1561


In [3]:
df.shape

(12923, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12923 entries, 0 to 12922
Data columns (total 4 columns):
udid            12923 non-null int64
time            12923 non-null int64
count           12923 non-null int64
avg_duration    12923 non-null int64
dtypes: int64(4)
memory usage: 403.9 KB


In [5]:
df.describe()

,udid,time,count,avg_duration
count,1.292300e+04,1.292300e+04,12923.000000,12923.000000
mean,1.039821e+08,1.478318e+09,1.456009,3072.271377
std,6.143904e+05,2.132706e+06,1.462151,11164.270049
min,7.438516e+07,1.474589e+09,1.000000,1.000000
25%,1.040141e+08,1.476572e+09,1.000000,53.000000
50%,1.040233e+08,1.478135e+09,1.000000,259.000000
75%,1.040263e+08,1.480110e+09,1.000000,954.500000
max,1.040312e+08,1.482364e+09,36.000000,86253.000000


In [6]:
df['time']= pd.to_datetime(df['time'], unit='s')

In [7]:
df.head(10)

,udid,time,count,avg_duration
0,104008439,2016-09-23 00:00:00,2,112
1,104011104,2016-09-23 00:00:00,2,44
2,104022241,2016-09-23 00:00:00,2,413
3,104022356,2016-09-23 00:00:00,2,125
4,104022069,2016-09-23 01:00:00,5,1561
5,104022090,2016-09-23 01:00:00,3,2631
6,104022326,2016-09-23 01:00:00,2,70
7,103995348,2016-09-23 01:00:00,1,72612
8,104022332,2016-09-23 01:00:00,1,63
9,104009608,2016-09-23 01:00:00,2,639


In [8]:
df['time'].max()

Timestamp('2016-12-21 23:40:00')

In [9]:
df_active=df[df['time']>'2016-12-08']

In [10]:
df['avg_duration'].mean()

3072.271376615337

In [11]:
df_active.describe()

,udid,count,avg_duration
count,1.603000e+03,1603.000000,1603.000000
mean,1.039632e+08,1.436057,3194.339988
std,6.322460e+05,1.538559,11872.346295
min,9.213756e+07,1.000000,1.000000
25%,1.040128e+08,1.000000,51.000000
50%,1.040269e+08,1.000000,259.000000
75%,1.040303e+08,1.000000,921.500000
max,1.040312e+08,30.000000,86253.000000


In [12]:
df_active['avg_duration'].mean()

3194.3399875233936

In [13]:
df_active['avg_duration'].describe()

count     1603.000000
mean      3194.339988
std      11872.346295
min          1.000000
25%         51.000000
50%        259.000000
75%        921.500000
max      86253.000000
Name: avg_duration, dtype: float64

In [14]:
df_active.head()

,udid,time,count,avg_duration
11320,104023758,2016-12-08 00:50:00,1,12
11321,103987642,2016-12-08 01:00:00,1,514
11322,104023758,2016-12-08 01:00:00,2,196
11323,104030129,2016-12-08 01:00:00,2,12933
11324,104023758,2016-12-08 01:10:00,2,908


In [15]:
df_active.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1603 entries, 11320 to 12922
Data columns (total 4 columns):
udid            1603 non-null int64
time            1603 non-null datetime64[ns]
count           1603 non-null int64
avg_duration    1603 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.6 KB


In [16]:
active = df_active[['udid', 'count','avg_duration']].groupby(['udid'], as_index=False).sum().sort_values(by='count', ascending=False)

In [17]:
active.insert(loc=len(active.columns), column='churn',value = 1)

In [18]:
active.head()
active.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 19 to 743
Data columns (total 4 columns):
udid            744 non-null int64
count           744 non-null int64
avg_duration    744 non-null int64
churn           744 non-null int64
dtypes: int64(4)
memory usage: 29.1 KB


In [19]:
players = df[['udid', 'count','avg_duration']].groupby(['udid'], as_index=False).sum().sort_values(by='count', ascending=False)

In [20]:
players.insert(loc=len(players.columns), column='churn', value=2)
players.head()
players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4267 entries, 139 to 4266
Data columns (total 4 columns):
udid            4267 non-null int64
count           4267 non-null int64
avg_duration    4267 non-null int64
churn           4267 non-null int64
dtypes: int64(4)
memory usage: 166.7 KB


In [21]:
players.append(active, ignore_index=True)

,udid,count,avg_duration,churn
0,103972991,542,217370,2
1,104023758,381,161146,2
2,104012689,257,207242,2
3,104020784,257,222063,2
4,104021079,242,19624,2
5,104025917,112,49378,2
6,104021384,108,62747,2
7,104022298,99,59295,2
8,104022069,98,35845,2
9,104015862,96,67173,2


In [25]:
players[['udid', 'churn']].groupby(['udid'], as_index=False).sum().sort_values(by='churn', ascending=False)

,udid,churn
0,74385162,2
2849,104025684,2
2835,104025632,2
2836,104025637,2
2837,104025640,2
2838,104025643,2
2839,104025647,2
2840,104025648,2
2841,104025649,2
2842,104025652,2
